<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/sk_whisper_large_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-cmsw9d6c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-cmsw9d6c
  Resolved https://github.com/huggingface/transformers to commit 510270af34994c24f2f8792d2afd74114c70d736
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8076097 sha256=18d4e928bb92b4f042cc79287f32e60acec493d1b324dacc2de3db989c514f3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-1p03gpyq/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [5]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "sk", split="validation")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 696
})

In [7]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=e7747ca428df7c3e1ab4b74f996cdbf190362eeb9ca1abadcf21037bc13162a7
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [8]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:56<00:00, 54.4MiB/s]


In [9]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [10]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [11]:
text

,0
0,vážené dámy a páni nie je možné odpovedať na v...
1,cieľ digitálnej agendy dosiahnuť do roku two t...
2,ak bude menej byrokracie menej papierov neodra...
3,bol som svedkom komplikovaných hraničných kont...
4,je mi veľmi ľúto ak to dospeje k tomu že veľká...
...,...
691,tak aby aj ľudia so zdravotným postihnutím kto...
692,budem rada ak sa nám podarí presadiť aby sme v...
693,považujem za potrebné aby smernica prísne dodr...
694,potvrdzujem pán predseda európskeho parlamentu...


In [12]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [13]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,vážené dámy a páni nie je možné odpovedať na v...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00030517578125, -0.000396728515625, -0.0017...",16000
1,male,cieľ digitálnej agendy dosiahnuť do roku two t...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00152587890625, 0.0018310546875, 0.00225830...",16000
2,male,ak bude menej byrokracie menej papierov neodra...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001861572265625, 0.002532958984375, -0.0006...",16000
3,male,bol som svedkom komplikovaných hraničných kont...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00311279296875, -0.00152587890625, 0.00256...",16000
4,female,je mi veľmi ľúto ak to dospeje k tomu že veľká...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.06915283203125, -0.0721435546875, -0.07650...",16000
...,...,...,...,...,...
691,female,tak aby aj ľudia so zdravotným postihnutím kto...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.055755615234375, 0.10894775390625, 0.144317...",16000
692,female,budem rada ak sa nám podarí presadiť aby sme v...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0091552734375, 0.0045166015625, 0.000274658...",16000
693,male,považujem za potrebné aby smernica prísne dodr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.005859375, 0.00518798828125, 0.002960205078...",16000
694,male,potvrdzujem pán predseda európskeho parlamentu...,/root/.cache/huggingface/datasets/downloads/ex...,"[9.1552734375e-05, 0.00030517578125, 0.0008850...",16000


In [14]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,vážené dámy a páni nie je možné odpovedať na v...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00030517578125, -0.000396728515625, -0.0017...",16000
1,male,cieľ digitálnej agendy dosiahnuť do roku two t...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00152587890625, 0.0018310546875, 0.00225830...",16000
2,male,ak bude menej byrokracie menej papierov neodra...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001861572265625, 0.002532958984375, -0.0006...",16000
3,male,bol som svedkom komplikovaných hraničných kont...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00311279296875, -0.00152587890625, 0.00256...",16000
5,male,len si predstavme že by sme naozaj mohli vydať...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.042388916015625, -0.04571533203125, -0.047...",16000
...,...,...,...,...,...
689,male,občania slovenskej republiky ktorí si už pomal...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.0001220703125, 0.000213623046875, 0.00...",16000
690,male,máme rovnaký jazyk.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00213623046875, -0.0028076171875, -0.00332...",16000
693,male,považujem za potrebné aby smernica prísne dodr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.005859375, 0.00518798828125, 0.002960205078...",16000
694,male,potvrdzujem pán predseda európskeho parlamentu...,/root/.cache/huggingface/datasets/downloads/ex...,"[9.1552734375e-05, 0.00030517578125, 0.0008850...",16000


In [15]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
4,female,je mi veľmi ľúto ak to dospeje k tomu že veľká...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.06915283203125, -0.0721435546875, -0.07650...",16000
9,female,a chcem vás ubezpečiť že rovnako to cítia aj v...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00054931640625, 0.002197265625, 0.00320434...",16000
13,female,dovoľte mi najprv krátko vysvetliť dôvody ktor...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.006378173828125, -0.006011962890625, -0.00...",16000
15,female,sú to skutočne reálne spoločnosti alebo ide o ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.015960693359375, 0.009613037109375, -0.0233...",16000
16,female,aleže sa množia prípady podstatných rozdielov ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0003662109375, -0.0023193359375, -0.004272...",16000
...,...,...,...,...,...
685,female,a na otázku či aj asistent mladého so zdravotn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00115966796875, -0.002410888671875, 0.00030...",16000
687,female,druhým dôvodom mojej podpory je skutočnosť že ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 6.103515625e-05, 0.0001525878...",16000
688,female,je povinnosťou komisie ako strážcu zmlúv aby k...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000732421875, 0.001983642578125, 0.0025329...",16000
691,female,tak aby aj ľudia so zdravotným postihnutím kto...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.055755615234375, 0.10894775390625, 0.144317...",16000


In [16]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="sk")
    return result["text"]

In [17]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [18]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.35714285714285715
Processed 2 files, WER: 0.23529411764705882
Processed 3 files, WER: 0.2222222222222222
Processed 4 files, WER: 0.5
Processed 5 files, WER: 0.2
Processed 6 files, WER: 0.2727272727272727
Processed 7 files, WER: 0.20833333333333334
Processed 8 files, WER: 0.5
Processed 9 files, WER: 0.28
Processed 10 files, WER: 0.13793103448275862
Processed 11 files, WER: 0.28125
Processed 12 files, WER: 0.1875
Processed 13 files, WER: 0.25
Processed 14 files, WER: 0.19607843137254902
Processed 15 files, WER: 0.16666666666666666
Processed 16 files, WER: 0.34615384615384615
Processed 17 files, WER: 0.2
Processed 18 files, WER: 0.3333333333333333
Processed 19 files, WER: 0.2727272727272727
Processed 20 files, WER: 0.19047619047619047
Processed 21 files, WER: 0.2857142857142857
Processed 22 files, WER: 0.4827586206896552
Processed 23 files, WER: 0.18421052631578946
Processed 24 files, WER: 0.12
Processed 25 files, WER: 0.0
Processed 26 files, WER: 0.125
Processed

In [20]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.3
Processed 2 files, WER: 0.23076923076923078
Processed 3 files, WER: 0.3246753246753247
Processed 4 files, WER: 0.3125
Processed 5 files, WER: 0.16666666666666666
Processed 6 files, WER: 0.5
Processed 7 files, WER: 0.1
Processed 8 files, WER: 0.2
Processed 9 files, WER: 0.26666666666666666
Processed 10 files, WER: 0.25
Processed 11 files, WER: 0.18518518518518517
Processed 12 files, WER: 0.5833333333333334
Processed 13 files, WER: 0.5833333333333334
Processed 14 files, WER: 0.12903225806451613
Processed 15 files, WER: 0.225
Processed 16 files, WER: 0.21739130434782608
Processed 17 files, WER: 0.1935483870967742
Processed 18 files, WER: 0.3333333333333333
Processed 19 files, WER: 0.1
Processed 20 files, WER: 0.0
Processed 21 files, WER: 0.05555555555555555
Processed 22 files, WER: 0.13043478260869565
Processed 23 files, WER: 0.3448275862068966
Processed 24 files, WER: 0.1
Processed 25 files, WER: 0.5555555555555556
Processed 26 files, WER: 0.21052631578947367
